In [12]:
import os
import bisect
import itertools
import math
import numpy as np
import pandas as pd
import requests
import pickle
from datetime import datetime as dt
from datetime import timedelta
import pytz
from pytz import timezone
from pprint import pprint
import pandas_datareader.data as web
from collections import namedtuple
from operator import itemgetter

import cufflinks as cf
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import yfinance as yf
import fundamentalanalysis as fa
import streamlit as st

import holidays
import pandas_datareader.data as web
import yahoo_fin.stock_info as si
from urllib.request import Request, urlopen
from html_table_parser.parser import HTMLTableParser 
import streamlit as st

from load_data import *
from get_data import get_tickers

pd.options.plotting.backend = "plotly"
# pd.set_option('display.max_rows', None)


In [13]:
rf_rates = get_rf_data()
SPY_df = get_SPY_data()
SPY_info_df = get_SPY_info()
ticker_info = get_ticker_info()
ticker_list = SPY_info_df.index.to_list()
sector_list = SPY_info_df['Sector'].unique()
first_date = SPY_df.iloc[0].name
last_date = SPY_df.iloc[-1].name
yr_ago = last_date - timedelta(days=365)
# TTM_ratios, ratios_data_report = load_TTM_ratios()

## Technical Analysis

To Do:
- Peak-Troughs / Trendlines
- MAs
- Volume Analysis

### Peak-Troughs / Trendlines

Maybe use linreg gradient to determine primary and secondary trend

In [20]:
from scipy.signal import find_peaks, peak_prominences, argrelextrema
from sklearn import preprocessing

pd.options.display.float_format = "{:,.2f}".format

In [21]:
end = last_date
years = 1
days = 180
start = end - timedelta(days * years)
ticker = 'a'
df = get_ticker_data(ticker)[start:end]
df_5m = get_interval_market_data(ticker, '5 Min')
open = df['Open'] 
high = df['High']
low = df['Low']
close = df['Close']
volume = df['Volume']

# start = end - timedelta(20)
# print(divmod((end - start).days, 365))
print(f'start: {start.date()}, end: {end.date()}')

start: 2022-09-01, end: 2023-02-28


In [22]:
p = find_peaks(close)[0]
t = find_peaks(close * -1)[0]
n = len(min(p, t, key=len))
pprominences = peak_prominences(close, p)[0]
pheight = close[p[:n]].to_numpy() - pprominences[:n]
tprominences = peak_prominences(close * -1, t)[0]
theight = close[t[:n]].to_numpy() + tprominences[:n]

for i in range(len(min(p, t, key=len))):
    # if i > 0:
    # if i % 2 == 0:
    # else:
    print(f'peak - dst = trough || {close[p[i]]:.2f} - {pprominences[i]:.2f} = {pheight[i]:.2f}')
    print(f'trough + dst = peak || {close[t[i]]:.2f} + {tprominences[i]:.2f} = {theight[i]:.2f}')

peak - dst = trough || 139.87 - 11.86 = 128.01
trough + dst = peak || 128.01 + 0.92 = 128.93
peak - dst = trough || 130.72 - 0.83 = 129.89
trough + dst = peak || 129.89 + 0.83 = 130.72
peak - dst = trough || 123.72 - 2.11 = 121.61
trough + dst = peak || 121.61 + 2.11 = 123.72
peak - dst = trough || 132.64 - 7.00 = 125.64
trough + dst = peak || 121.55 + 18.32 = 139.87
peak - dst = trough || 127.90 - 2.20 = 125.70
trough + dst = peak || 125.64 + 7.00 = 132.64
peak - dst = trough || 132.30 - 6.36 = 125.94
trough + dst = peak || 125.70 + 2.20 = 127.90
peak - dst = trough || 137.69 - 1.20 = 136.49
trough + dst = peak || 125.94 + 6.36 = 132.30
peak - dst = trough || 138.77 - 0.42 = 138.35
trough + dst = peak || 136.49 + 1.20 = 137.69
peak - dst = trough || 140.89 - 6.43 = 134.46
trough + dst = peak || 138.35 + 0.42 = 138.77
peak - dst = trough || 138.75 - 1.75 = 137.00
trough + dst = peak || 134.46 + 6.43 = 140.89
peak - dst = trough || 148.31 - 3.87 = 144.44
trough + dst = peak || 137.00 + 

In [23]:
dt.strftime(end, '%d/%m/%y %H:%M')

'28/02/23 00:00'

In [24]:
peaks = argrelextrema(close.to_numpy(), np.greater)[0]
troughs = argrelextrema(close.to_numpy(), np.less)[0]
print(f'peaks: {len(peaks)}, troughs: {len(troughs)}')
peaks_dates = [dt.strftime(df.iloc[i].name, '%d-%m-%y') for i in peaks]
troughs_dates = [dt.strftime(df.iloc[i].name, '%d-%m-%y') for i in troughs]
print('Peaks:\n', peaks_dates)
print('Troughs:\n', troughs_dates)

peaks: 31, troughs: 31
Peaks:
 ['12-09-22', '19-09-22', '28-09-22', '05-10-22', '13-10-22', '18-10-22', '26-10-22', '28-10-22', '01-11-22', '08-11-22', '11-11-22', '15-11-22', '18-11-22', '22-11-22', '25-11-22', '01-12-22', '08-12-22', '13-12-22', '21-12-22', '27-12-22', '29-12-22', '05-01-23', '11-01-23', '13-01-23', '23-01-23', '26-01-23', '02-02-23', '07-02-23', '13-02-23', '23-02-23', '27-02-23']
Troughs:
 ['02-09-22', '16-09-22', '27-09-22', '30-09-22', '11-10-22', '14-10-22', '20-10-22', '27-10-22', '31-10-22', '03-11-22', '09-11-22', '14-11-22', '17-11-22', '21-11-22', '23-11-22', '29-11-22', '06-12-22', '09-12-22', '19-12-22', '22-12-22', '28-12-22', '30-12-22', '09-01-23', '12-01-23', '19-01-23', '24-01-23', '30-01-23', '06-02-23', '09-02-23', '22-02-23', '24-02-23']


In [36]:
def peaks_valleys_analysis(df):
    df = df.copy().drop(columns='Adj Close')
    close = df['Close']
    peaks = argrelextrema(close.to_numpy(), np.greater)[0]
    valleys = argrelextrema(close.to_numpy(), np.less)[0]
    df['isPeak'] = 0
    df['isValley'] = 0
    df['PV Trend'] = 0
    last_row = df.shape[0] - 1
    PVs = list(itertools.zip_longest(peaks, valleys, fillvalue=last_row))
    first = 'peaks' if min(peaks[0], valleys[0]) == peaks[0] else 'valleys'
    m, n = (0, 1) if first == 'peaks' else (1, 0)

    for i, (p, v) in enumerate(PVs):
        p_ix = df.iloc[p].name
        v_ix = df.iloc[v].name
        df.loc[p_ix, 'isPeak'] = 1
        df.loc[v_ix, 'isValley'] = 1
        df.loc[p_ix, 'Peak'] = close[p]
        df.loc[v_ix, 'Valley'] = close[v]
        if i > 0:
            a, b = (i, i - 1) if i % 2 != 0 else (i, i)
            pp = peaks[i - 1]
            pv = valleys[i - 1]
            d2 = abs(close[PVs[a][m]] - close[PVs[b][n]])
            d1 = abs(close[pv] - close[pp])
    
            print(f'{i}: {close[PVs[a][m]]:.2f} - {close[PVs[b][n]]:.2f} = {d2:.2f},\
                   \nRetraced: {d2:.2f} / {d1:.2f} = {d2/d1:.2f}\n')
            
            m, n = n, m
            d_ix1 = df.iloc[max(PVs[i - 1])].name
            d_ix2 = df.iloc[min(PVs[i])].name
            df.loc[d_ix1, 'PV Dist'] = d1
            df.loc[d_ix2, 'PV Dist'] = d2
            df.loc[d_ix2, 'Retraced'] = d2 / d1
            if close[p] - close[pp] >= 0 and close[v] - close[pv] >= 0:
                trend = 1 # uptrend
            elif close[p] - close[pp] < 0 and close[v] - close[pv] < 0:
                trend = -1 # downtrend
            else:
                trend = 0
            start = df.iloc[min(pp, pv)].name
            end = df.iloc[max(p, v)].name
            df.loc[start:end, 'PV Trend'] = trend
            # print(f"Peak:   {dt.strftime(df.iloc[pp].name, '%d-%m-%y')} - {dt.strftime(p_ix, '%d-%m-%y')} " \
            #   f"|| {close[pp]:,.2f} - {close[p]:,.2f},\n" \
            #   f"Valley: {dt.strftime(df.iloc[pv].name, '%d-%m-%y')} - {dt.strftime(v_ix, '%d-%m-%y')} " \
            #   f"|| {close[pv]:,.2f} - {close[v]:,.2f},\n" \
            #   f"Trend: {trend}\n")

    # CAN USE CHANGES IN TREND TO IDENTIFY POINTS TO DRAW TRENDLINES BTWN
    df['PV Changepoint'] = df['PV Trend'].diff()
    ixs = df[df['PV Changepoint'] != 0].index.to_list()
    trendlines = []
    
    for i, ix in enumerate(ixs[1:], 1):
        start = ixs[i - 1]
        end = ix
        npeaks = df.loc[start:end, 'isPeak'].sum()
        nvalleys = df.loc[start:end, 'isValley'].sum()
        indices = []
        if df.loc[ix, 'PV Changepoint'] < 0 and npeaks > 2:
            indices = df[df['isPeak'] != 0][start:end].index.to_list()
        if df.loc[ix, 'PV Changepoint'] > 0 and nvalleys > 2:
            indices = df[df['isValley'] != 0][start:end].index.to_list()
        if indices:    
            x1, x2 = indices[0], indices[-1]
            y1, y2 = close[x1], close[x2]
            trendlines.append(((x1, x2), (y1, y2)))
        # print(f"{i}.".ljust(3), f"{dt.strftime(start, '%d-%m-%y')} - {dt.strftime(end, '%d-%m-%y')}, peaks: {peaks.sum()}, valleys: {valleys.sum()}")
    
    cols = ['isPeak', 'isValley', 'Peak', 'Valley', 'PV Dist', 'Retraced', 'PV Changepoint']
    df.drop(columns=cols, inplace=True)
    
    return df, peaks, valleys, trendlines

In [37]:
pv_df, peaks, valleys, trendlines = peaks_valleys_analysis(df)
# print(trendlines)
pv_df.loc[:'31-10-22', pv_df.columns[-5:]]

1: 129.89 - 139.87 = 9.98,                   
Retraced: 9.98 / 11.86 = 0.84

2: 123.72 - 121.61 = 2.11,                   
Retraced: 2.11 / 0.83 = 2.54

3: 121.55 - 123.72 = 2.17,                   
Retraced: 2.17 / 2.11 = 1.03

4: 127.90 - 125.64 = 2.26,                   
Retraced: 2.26 / 11.09 = 0.20

5: 125.70 - 127.90 = 2.20,                   
Retraced: 2.20 / 2.26 = 0.97

6: 137.69 - 125.94 = 11.75,                   
Retraced: 11.75 / 6.60 = 1.78

7: 136.49 - 137.69 = 1.20,                   
Retraced: 1.20 / 11.75 = 0.10

8: 140.89 - 138.35 = 2.54,                   
Retraced: 2.54 / 2.28 = 1.11

9: 134.46 - 140.89 = 6.43,                   
Retraced: 6.43 / 2.54 = 2.53

10: 148.31 - 137.00 = 11.31,                   
Retraced: 11.31 / 4.29 = 2.64

11: 146.38 - 148.31 = 1.93,                   
Retraced: 1.93 / 11.31 = 0.17

12: 146.19 - 144.44 = 1.75,                   
Retraced: 1.75 / 1.62 = 1.08

13: 145.14 - 146.19 = 1.05,                   
Retraced: 1.05 / 1.75 = 0.60



,High,Low,Close,Volume,PV Trend
2022-09-01,129.11,126.21,128.93,1282000,0
2022-09-02,130.27,127.12,128.01,1695900,0
2022-09-06,130.57,127.72,129.30,1533500,0
2022-09-07,131.82,128.61,131.43,1254600,0
2022-09-08,135.27,130.71,135.19,1355100,0
2022-09-09,137.92,135.43,137.63,2425200,0
2022-09-12,140.00,137.23,139.87,1851100,0
2022-09-13,137.41,133.44,133.54,1587000,0
2022-09-14,135.36,132.10,133.25,1577000,0
2022-09-15,135.51,132.66,133.16,1446500,0


In [9]:
f = 'data/financial_ratios/Annual'
ticker = 'A'
file = ticker + '.csv'
df = pd.read_csv(os.path.join(f, file))
not_current = []
if not df.empty and str(dt.now().year - 1) != df.columns[1]:
    not_current.append(ticker)

len(not_current)

0

In [11]:
df.columns[0]

'Unnamed: 0'

In [202]:
# peaks = height_ixs
# fig = go.Figure()
# fig.add_scatter(x=df.index, y=df['rtn'], name='Return', line_color='red', opacity=0.5)
# fig.add_scatter(x=df.index, y=df['log_rtn'], name='Log Return', line_color='blue', opacity=0.5)
# y = scaled_df['Close']
cs = go.Candlestick(x=df.index, 
                    open=df['Open'], 
                    high=df['High'],
                    low=df['Low'], 
                    close=df['Close'],
                    name=ticker)
cs.increasing.fillcolor = 'green'
cs.increasing.line.color = 'darkgreen'
cs.decreasing.fillcolor = 'red'
cs.decreasing.line.color = 'indianred'
fig = go.Figure()
    
fig.add_scatter(x=pv_df.index, 
                y=pv_df.Close, 
                name='Close', 
                mode='lines',
                line_color='blue', 
                opacity=0.5, 
                connectgaps=True)
fig.add_scatter(x=pv_df.Close[peaks].index,
                y=pv_df.Close[peaks],
                name='Peaks',
                mode='markers',
                marker=dict(symbol='x', color='orange'), 
                opacity=0.5)
fig.add_scatter(x=pv_df.Close[valleys].index,
                y=pv_df.Close[valleys],
                name='Valleys',
                mode='markers',
                marker=dict(symbol='x', color='red'), 
                opacity=0.5)

for x, y in trendlines:
    fig.add_scatter(x=x,
                    y=y,
                    name='Trend',
                    mode='lines', 
                    opacity=0.5,
                    showlegend=False)
# fig.add_scatter(x=close[peak_troughs].index,
#                 y=close[peak_troughs],
#                 name='peak_troughs',
#                 mode='markers',
#                 marker=dict(symbol='circle-open', color='green', size=8))
# fig.add_vline(x=close[peaks].index,
#                 # y=heights,
#                 # mode='lines',
#                 )
us_holidays = list(holidays.US(range(start.year, end.year + 1)).keys())
rangebreaks = [dict(bounds=["sat", "mon"]), dict(values=us_holidays)]
# rangebreaks.extend([dict(bounds=[16, 9.5], pattern="hour")])
fig.update_xaxes(rangebreaks=rangebreaks)
fig.update_layout(hovermode="x unified", paper_bgcolor='gray', plot_bgcolor='gray')
fig

### S/R Levels

In [14]:
from sklearn import preprocessing
import time
# pd.set_option('display.max_rows', None)
pd.options.display.float_format = "{:,.2f}".format

In [15]:
end = last_date
years = 1
days = 180
start = end - timedelta(days * years)
ticker = 'aapl'
df = get_ticker_data(ticker)[start:end]

In [16]:
def fibonacci_retracement_levels(df):
    highest_swing = -1
    lowest_swing = -1
    high = df['High']
    low = df['Low']
    nr = df.shape[0]

    for i in range(1, nr - 1):
        if high[i] > high[i - 1] and high[i] > high[i + 1] \
            and (highest_swing == -1 or high[i] > high[highest_swing]):
            highest_swing = i
        if low[i] < low[i - 1] and low[i] < low[i + 1] \
            and (lowest_swing == -1 or low[i] < low[lowest_swing]):
            lowest_swing = i

    ratios = [0, 0.236, 0.382, 0.5, 0.618, 0.786, 1]
    levels = []
    max_level = high[highest_swing]
    min_level = low[lowest_swing]

    for ratio in ratios:
        # Uptrend
        if highest_swing > lowest_swing:
            level = max_level - (max_level - min_level) * ratio
        # Downtrend
        else:
            level = min_level + (max_level - min_level) * ratio
        levels.append(level)

    frl_data = {}
    unit = 'minutes' if df.iloc[0].name.minute != 0 else 'days'  
    prev_date = df.iloc[0].name

    for i in range(nr):
        date = df.iloc[i].name
        s_date = date.strftime('%d-%m-%y')
        l = bisect.bisect_left(levels, low[i])
        h = bisect.bisect_left(levels, high[i])
        h = h - 1 if h > 0 else h
        frls = [levels[h], levels[l]] if h != l else [levels[h]]
        # Deals with multiple levels being tested by one bar
        for frl in frls:
            if high[i] > frl and low[i] < frl:
                cum_vol = df.loc[prev_date:date, 'Volume'].sum()
                delta = date - prev_date # time it takes level to form
                delta = delta.days if unit == 'days' else delta.total_seconds() / 60
                prev_date = date
                d = {'Date': [], 'Timedelta': [], 'Volume': [], 'Tested': 0}
                frl_data.setdefault(frl, d)
                frl_data[frl]['Date'].append(s_date)
                frl_data[frl]['Timedelta'].append(delta)
                frl_data[frl]['Volume'].append(cum_vol)
                frl_data[frl]['Tested'] += 1             

    # Calculate significance of levels       
    d = {'FRL': [], 'Date': [], 'Timedelta': [], 'Volume': [], 'Tested': []}

    for k, v in frl_data.items():
        d['FRL'].append(k)
        d['Volume'].append(sum(v['Volume']))
        d['Timedelta'].append(sum(v['Timedelta']))
        d['Tested'].append(v['Tested'])
        d['Date'].append(v['Date'][-1])
        # print(v['Date'][-1])

    # NOT YET DECIDED HOW TO CALCULATE FRL SIGNAL

    return ratios, levels, frl_data


In [17]:
r, l, d = fibonacci_retracement_levels(df)
d

{155.68073529052734: {'Date': ['02-09-22',
   '06-09-22',
   '07-09-22',
   '08-09-22',
   '09-09-22',
   '13-09-22',
   '14-09-22',
   '20-09-22',
   '21-09-22',
   '28-10-22',
   '03-02-23',
   '16-02-23'],
  'Timedelta': [0, 4, 1, 1, 1, 4, 1, 4, 1, 1, 1, 13],
  'Volume': [76957800,
   150672600,
   161164400,
   172373400,
   152952600,
   295641400,
   210622000,
   351442800,
   209386600,
   273942600,
   272618900,
   742740900],
  'Tested': 12},
 148.9456159057617: {'Date': ['16-09-22',
   '23-09-22',
   '28-09-22',
   '24-10-22',
   '26-10-22',
   '27-10-22',
   '28-10-22',
   '02-11-22',
   '11-11-22',
   '14-11-22',
   '15-11-22',
   '16-11-22',
   '17-11-22',
   '21-11-22',
   '22-11-22',
   '01-12-22',
   '05-12-22',
   '13-12-22',
   '02-02-23',
   '03-02-23',
   '21-02-23',
   '22-02-23',
   '23-02-23',
   '27-02-23',
   '28-02-23'],
  'Timedelta': [2,
   2,
   5,
   3,
   2,
   1,
   0,
   5,
   1,
   3,
   1,
   1,
   1,
   4,
   1,
   1,
   4,
   1,
   1,
   0,
   5,


In [18]:
def isSupport(df, i):
    '''Returns True if value is a price support level'''

    X = df['Low']
    support = X[i] < X[i - 1] \
                and X[i] < X[i + 1] \
                and X[i + 1] < X[i + 2] \
                and X[i - 1] < X[i - 2]

    return support


def isResistance(df, i):
    '''Returns True if value is a price resistance level'''

    X = df['High']
    resistance = X[i] > X[i - 1] \
                    and X[i] > X[i + 1] \
                    and X[i + 1] > X[i + 2] \
                    and X[i - 1] > X[i - 2] 

    return resistance
    

In [19]:
def convert_to_timestamp(x):
    """Convert date objects to integers"""
    
    return time.mktime(x.timetuple())

In [20]:
def sr_levels(df):
    '''Returns key support/resistance levels for a security'''

    df = df.copy()
    df['SR Signal'] = 0
    unit = 'minutes' if df.iloc[0].name.minute != 0 else 'days'  
    spt = 0
    rst = 0
    levels = []
    s_levels = []
    prev_date = df.iloc[0].name
    sr_data = {}
    many_tests = {} # dict of bars that test more than 1 level
    s = (df['High'] - df['Low']).mean()

    def isFarFromLevel(l):
        '''Returns True if price is not near a previously discovered support or resistance'''
        
        return np.sum([abs(l - x[1]) < s for x in levels]) == 0

    for i in range(2, df.shape[0] - 2):
        date = df.iloc[i].name
        s_date = date.strftime('%d-%m-%y')
        high = df['High'][i]
        low = df['Low'][i]
        close = df['Close'][i]
        new_spt = False
        new_rst = False
        sr_switch = False

        if isSupport(df, i):
            if isFarFromLevel(low):
                new_spt = True
                spt = low
                df.loc[date, 'Support'] = spt
                levels.append((i, spt))
                s_levels = sorted([x[1] for x in levels])
                print('NS'.ljust(5), f'- {date.date()} - S: {spt:.2f}, R: {rst:.2f}, hi: {high:.2f}, lo: {low:.2f}')
        
        if isResistance(df, i):
            if isFarFromLevel(high):
                new_rst = True
                rst = high
                df.loc[date, 'Resistance'] = rst
                levels.append((i, rst))
                s_levels = sorted([x[1] for x in levels])
                print('NR'.ljust(5), f'- {date.date()} - R: {rst:.2f}, S: {spt:.2f}, hi: {high:.2f}, lo: {low:.2f},')    

        # Switch support to resistance & vice versa
        if len(levels) > 1:
            if new_spt:
                ix = bisect.bisect(s_levels, spt)
                rst = s_levels[ix] if ix < len(s_levels) else s_levels[ix - 1]
            if new_rst:
                ix = bisect.bisect_left(s_levels, rst)
                spt = s_levels[ix - 1] if ix > 0 else s_levels[ix]
            if low > rst: # When resistance broken 
                sr_switch = True
                spt = rst
                ix = bisect.bisect(s_levels, low)
                rst = s_levels[ix] if ix < len(s_levels) else s_levels[ix - 1]
                print('R-S'.ljust(5), f'- {date.date()} - S: {spt:.2f}, R: {rst:.2f}, hi: {high:.2f}, lo: {low:.2f}')
            if high < spt: # When support broken 
                sr_switch = True
                rst = spt
                ix = bisect.bisect_left(s_levels, high)
                spt = s_levels[ix - 1] if ix > 0 else s_levels[ix]
                print('S-R'.ljust(5), f'- {date.date()} - R: {rst:.2f}, S: {spt:.2f}, hi: {high:.2f}, lo: {low:.2f}')
        
        if new_rst or new_spt or sr_switch:
            cum_vol = df.loc[prev_date:date, 'Volume'].sum()
            delta = date - prev_date # time it takes level to form
            delta = delta.days if unit == 'days' else delta.total_seconds() / 60
            prev_date = date
            d = {'Date': [], 'Timedelta': [], 'Volume': [], 'SR': [], 'Tested': 0, 'Tested Date': []}
            sr_data.setdefault(spt, d)
            sr_data[spt]['Date'].append(s_date)
            sr_data[spt]['Timedelta'].append(delta)
            sr_data[spt]['Volume'].append(cum_vol)
            sr_data[spt]['SR'].append('S')
            # Prevents double-counting when lowest/highest level is both support & resistance
            if spt != rst:
                sr_data.setdefault(rst, d)
                sr_data[rst]['Date'].append(s_date)
                sr_data[rst]['Timedelta'].append(delta)
                sr_data[rst]['Volume'].append(cum_vol)
                sr_data[rst]['SR'].append('R')
                                  
        if spt:
            if close < spt:
                df.loc[date, 'SR Signal'] = 1 # Generate signal
            # Check if S/R levels are tested       
            if high > spt and low < spt:
                sr_data[spt]['Tested'] += 1
                sr_data[spt]['Tested Date'].append(s_date)
                print('ST'.ljust(5), f'- {date.date()} - S: {spt:.2f}, R: {rst:.2f}, hi: {high:.2f}, lo: {low:.2f}')
                ix = bisect.bisect_left(s_levels, spt)
                n_spt = s_levels[ix - 1] if ix > 0 else s_levels[ix]    
                while low < n_spt and spt != rst and spt != n_spt:
                    print(f'SH-SL - {date.date()} - NS: {n_spt:.2f}, S: {spt:.2f}, R: {rst:.2f}, hi: {high:.2f}, lo: {low:.2f}')
                    rst = spt
                    spt = n_spt
                    many_tests.setdefault(i, set()).union([spt, rst])
                    if ix > 0:
                        ix -= 1
                        n_spt = s_levels[ix]
                        sr_data[n_spt]['Date'].append(s_date)
                        sr_data[n_spt]['Timedelta'].append(delta)
                        sr_data[n_spt]['Volume'].append(cum_vol)
                        sr_data[n_spt]['SR'].append('S')            
   
        if rst:
            if close > rst:
                df.loc[date, 'SR Signal'] = 1 # Generate signal
            # Check if S/R levels are tested       
            if high > rst and low < rst:
                if spt != rst: # Prevents double-counting
                    sr_data[rst]['Tested'] += 1
                    sr_data[rst]['Tested Date'].append(s_date)
                    print('RT'.ljust(5), f'- {date.date()} - R: {rst:.2f}, S: {spt:.2f}, hi: {high:.2f}, lo: {low:.2f}')
                    ix = bisect.bisect(s_levels, rst)
                    n_rst = s_levels[ix] if ix < len(s_levels) else s_levels[ix - 1]
                    while high > n_rst and spt != rst and rst != n_rst:
                        print(f'RL-RH - {date.date()} - NR: {n_rst:.2f}, R: {rst:.2f}, S: {spt:.2f}, hi: {high:.2f}, lo: {low:.2f}')
                        spt = rst
                        rst = n_rst
                        many_tests.setdefault(i, set()).union([spt, rst])
                        if ix < len(s_levels):
                            ix += 1
                            # print(f'ix: {ix}, {s_levels}')
                            n_rst = s_levels[ix]
                            sr_data[n_rst]['Date'].append(s_date)
                            sr_data[n_rst]['Timedelta'].append(delta)
                            sr_data[n_rst]['Volume'].append(cum_vol)
                            sr_data[n_rst]['SR'].append('R')        

        if spt and rst: 
            df.loc[date:, 'Support'] = spt
            df.loc[date:, 'Resistance'] = rst

    del sr_data[0]    

    # Calculate significance of levels       
    d = {'SR Level': [], 'Volume': [], 'Timedelta': [], 'Tested': [], 'Date': []}

    for k, v in sr_data.items():
        d['SR Level'].append(k)
        d['Volume'].append(sum(v['Volume']))
        d['Timedelta'].append(sum(v['Timedelta']))
        d['Tested'].append(v['Tested'])
        d['Date'].append(v['Date'][-1])
        # print(v['Date'][-1])

    ix = 'SR Level'
    sr_df = pd.DataFrame(d, index=d[ix]).drop(columns=ix)
    sr_df['Date'] = pd.to_datetime(sr_df['Date'])
    sr_df['Date'] = sr_df['Date'].apply(convert_to_timestamp)
    scaler = preprocessing.MinMaxScaler(feature_range=(1, 5))
    sd = scaler.fit_transform(sr_df)
    scaled_df = pd.DataFrame(sd, columns=sr_df.columns, index=d[ix])
    scaled_df['Signal'] = scaled_df.mean(axis=1)
    # print(sr_df)
    # print(scaled_df)
    
    # Make 'SR Signal' last column
    cols = list(df.columns)
    cols.append(cols.pop(cols.index('SR Signal')))
    df = df[cols]
    nr, nc = df.shape
    j = nc - 1 # 'SR Signal' column num

    for i in range(nr):
        if df['SR Signal'][i]:
            if i in many_tests:
                signal = 0
                for l in many_tests[i]:
                    signal += scaled_df.loc[l, 'Signal']
            else:
                if df['Close'][i] > df['Resistance'][i]:
                    l = df['Resistance'][i]
                if df['Close'][i] < df['Support'][i]:
                    l = df['Support'][i]
                signal = scaled_df.loc[l, 'Signal']
 
            df.iloc[i, j] = signal        
    
    return levels, df, sr_data

In [21]:
# sr_data.clear()
levels, ta_df, sr_data = sr_levels(df)
# levels

NS    - 2022-09-08 - S: 152.68, R: 0.00, hi: 156.36, lo: 152.68
NR    - 2022-09-12 - R: 164.26, S: 152.68, hi: 164.26, lo: 159.30,
ST    - 2022-09-15 - S: 152.68, R: 164.26, hi: 155.24, lo: 151.38
NS    - 2022-09-16 - S: 148.37, R: 164.26, hi: 151.35, lo: 148.37
RT    - 2022-09-19 - R: 152.68, S: 148.37, hi: 154.56, lo: 149.10
R-S   - 2022-09-20 - S: 152.68, R: 164.26, hi: 158.08, lo: 153.08
NR    - 2022-09-21 - R: 158.74, S: 152.68, hi: 158.74, lo: 153.60,
ST    - 2022-09-22 - S: 152.68, R: 158.74, hi: 154.47, lo: 150.91
S-R   - 2022-09-23 - R: 152.68, S: 148.37, hi: 151.47, lo: 148.56
RT    - 2022-09-26 - R: 152.68, S: 148.37, hi: 153.77, lo: 149.64
RT    - 2022-09-27 - R: 152.68, S: 148.37, hi: 154.72, lo: 149.95
ST    - 2022-09-28 - S: 148.37, R: 152.68, hi: 150.64, lo: 144.84
S-R   - 2022-09-29 - R: 148.37, S: 148.37, hi: 146.72, lo: 140.68
S-R   - 2022-09-30 - R: 148.37, S: 148.37, hi: 143.10, lo: 138.00
S-R   - 2022-10-03 - R: 148.37, S: 148.37, hi: 143.07, lo: 137.69
S-R   - 20

In [22]:
pprint(sr_data)

{134.3699951171875: {'Date': ['13-10-22',
                              '03-11-22',
                              '07-12-22',
                              '15-12-22',
                              '20-12-22',
                              '23-12-22',
                              '27-12-22',
                              '28-12-22',
                              '29-12-22',
                              '30-12-22',
                              '03-01-23',
                              '04-01-23',
                              '05-01-23',
                              '06-01-23',
                              '09-01-23',
                              '10-01-23',
                              '11-01-23',
                              '12-01-23',
                              '18-01-23'],
                     'SR': ['S',
                            'S',
                            'S',
                            'S',
                            'S',
                            'S',
   

In [79]:
d = {'SR Level': [], 'Volume': [], 'Timedelta': [], 'Tested': []}

for k, v in sr_data.items():
    d['SR Level'].append(k)
    d['Volume'].append(sum(v['Volume']))
    d['Timedelta'].append(sum(v['Timedelta']))
    d['Tested'].append(v['Tested'])

sr_df = pd.DataFrame(d, index=d['SR Level']).drop(columns='SR Level')
scaler = preprocessing.MinMaxScaler(feature_range=(1, 5))
sd = scaler.fit_transform(sr_df)
cols = ['Volume', 'Timedelta', 'Tested']
scaled_df = pd.DataFrame(sd, columns=sr_df.columns, index=d['SR Level'])
scaled_df['Signal'] = scaled_df.mean(axis=1)
print(sr_df.head(), '\n')
print(scaled_df)

            Volume  Timedelta  Tested
152.68  5601904700         78      16
164.26   609351400          8       0
148.37  7816926500         99      20
158.74  2146815800         31       0
134.37  4587985500         53       8 

        Volume  Timedelta  Tested  Signal
152.68    3.77       4.08    4.20    4.02
164.26    1.00       1.00    1.00    1.00
148.37    5.00       5.00    5.00    5.00
158.74    1.85       2.01    1.00    1.62
134.37    3.21       2.98    2.60    2.93
144.13    2.46       2.98    3.40    2.95


## MA Signals

- MA Crossovers, i.e. MA5/MA25, etc...
- Identify when moving averages converge

In [34]:
def calc_slope(x):
    slope = np.polyfit(range(len(x)), x, 1)[0] 
    
    return slope

In [98]:
def calculate_ma_signals(df, MAs):
    for i, ma in enumerate(MAs):
        ix = df.iloc[ma].name
        df[f'MA{ma}'] = df['Close'].rolling(ma).mean()
        df[f'MA{ma} Slope'] = df[f'Close'].rolling(ma).apply(calc_slope)
        # df[f'MA{ma} Slope Diff'] = df[f'MA{ma} Slope'].diff()
        df.loc[ix:, f'MA{ma} Signal'] = np.where(df['Close'][ma:] > df[f'MA{ma}'][ma:], 1, 0)
        df[f'MA{ma} Crossover'] = df[f'MA{ma} Signal'].diff()
        df.drop(columns=f'MA{ma} Signal', inplace=True)

    df.loc[:, 'MA Std. Dev'] = df.loc[:, [f'MA{ma}' for ma in MAs]].std(axis=1)
    
    return df

In [100]:
MAs = [10, 20, 50]
ma_df = calculate_ma_signals(ta_df, MAs)
# ma_df.loc['2022-09-12':'2022-11-12', ['MA10 Slope',	'MA10 Slope Diff', 'MA20 Slope', 'MA50 Slope']]
ma_df.loc['2022-09-12':'2022-11-12', ['MA10', 'MA20', 'MA50', 'MA Std. Dev']]

,MA10,MA20,MA50,MA Std. Dev
2022-09-12,157.70,NaN,NaN,NaN
2022-09-13,156.95,NaN,NaN,NaN
2022-09-14,156.59,NaN,NaN,NaN
2022-09-15,156.10,NaN,NaN,NaN
2022-09-16,155.38,NaN,NaN,NaN
2022-09-19,155.24,NaN,NaN,NaN
2022-09-20,155.48,NaN,NaN,NaN
2022-09-21,155.26,NaN,NaN,NaN
2022-09-22,155.09,NaN,NaN,NaN
2022-09-23,154.39,NaN,NaN,NaN


In [58]:
math.atan2(ma_df['MA10'].values, ma_df.index.values)

TypeError: only size-1 arrays can be converted to Python scalars

In [74]:
ma_df.loc['2022-10-03':'2022-10-04', ['MA10', 'MA10 Slope', 'MA20', 'MA20 Slope']]

,MA10,MA10 Slope,MA20,MA20 Slope
2022-10-03,148.93,-0.73,152.09,NaN
2022-10-04,147.85,-0.83,151.67,NaN


In [75]:
x = ma_df.loc['2022-10-03':'2022-10-04', 'MA10']
slope = np.polyfit(range(len(x)), x, 1)[0] 
slope


-1.0799987792968864

### Bollinger Bands

In [17]:
from talib import BBANDS, MACD, RSI

In [18]:
up, mid, down = BBANDS(df['Close'], timeperiod=21, nbdevup=2, nbdevdn=2, matype=0)
bands = {'BB Up': up, 'BB Mid': mid, 'BB Down': down}
bb_df = pd.DataFrame(bands)
bb_df

,BB Up,BB Mid,BB Down
2022-08-29,NaN,NaN,NaN
2022-08-30,NaN,NaN,NaN
2022-08-31,NaN,NaN,NaN
2022-09-01,NaN,NaN,NaN
2022-09-02,NaN,NaN,NaN
...,...,...,...
2023-02-17,159.14,148.58,138.01
2023-02-21,158.50,149.08,139.66
2023-02-22,158.18,149.45,140.73
2023-02-23,157.94,149.78,141.62


In [131]:
open.rolling(5, center=True).mean()

Datetime
2023-02-13 09:30:00-05:00      NaN
2023-02-13 09:35:00-05:00      NaN
2023-02-13 09:40:00-05:00   158.76
2023-02-13 09:45:00-05:00   158.66
2023-02-13 09:50:00-05:00   158.61
                             ...  
2023-02-16 15:40:00-05:00   160.52
2023-02-16 15:45:00-05:00   160.51
2023-02-16 15:50:00-05:00   160.51
2023-02-16 15:55:00-05:00      NaN
2023-02-16 16:00:00-05:00      NaN
Name: Open, Length: 313, dtype: float64

In [16]:
from talib import BBANDS, MACD, RSI

In [17]:
up, mid, low = BBANDS(df.Close, timeperiod=21, nbdevup=2, nbdevdn=2, matype=0)

In [21]:
# df[['BB Up', 'BB Mid', 'BB Down']] = up, mid, low
df['BB Up'] = up
df['BB Mid'] = mid
df['BB Down'] = low
df

,Open,High,Low,Close,Adj Close,Volume,BB Up,BB Mid,BB Down
Datetime,,,,,,,,,
2023-02-13 09:30:00-05:00,158.89,159.91,158.64,158.72,158.72,19034,NaN,NaN,NaN
2023-02-13 09:35:00-05:00,158.80,159.34,158.80,159.11,159.11,5547,NaN,NaN,NaN
2023-02-13 09:40:00-05:00,158.95,159.05,158.58,158.78,158.78,9484,NaN,NaN,NaN
2023-02-13 09:45:00-05:00,158.70,158.78,158.23,158.43,158.43,51620,NaN,NaN,NaN
2023-02-13 09:50:00-05:00,158.44,158.70,158.37,158.51,158.51,11596,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2023-02-16 15:40:00-05:00,160.51,160.70,160.24,160.67,160.67,32917,160.70,160.33,159.96
2023-02-16 15:45:00-05:00,160.66,160.69,160.16,160.31,160.31,52181,160.67,160.35,160.02
2023-02-16 15:50:00-05:00,160.35,160.85,160.34,160.59,160.59,79573,160.68,160.37,160.06


In [23]:
rsi = RSI(df.Close, timeperiod=14)
rsi.index

DatetimeIndex(['2023-02-13 09:30:00-05:00', '2023-02-13 09:35:00-05:00',
               '2023-02-13 09:40:00-05:00', '2023-02-13 09:45:00-05:00',
               '2023-02-13 09:50:00-05:00', '2023-02-13 09:55:00-05:00',
               '2023-02-13 10:00:00-05:00', '2023-02-13 10:05:00-05:00',
               '2023-02-13 10:10:00-05:00', '2023-02-13 10:15:00-05:00',
               ...
               '2023-02-16 15:15:00-05:00', '2023-02-16 15:20:00-05:00',
               '2023-02-16 15:25:00-05:00', '2023-02-16 15:30:00-05:00',
               '2023-02-16 15:35:00-05:00', '2023-02-16 15:40:00-05:00',
               '2023-02-16 15:45:00-05:00', '2023-02-16 15:50:00-05:00',
               '2023-02-16 15:55:00-05:00', '2023-02-16 16:00:00-05:00'],
              dtype='datetime64[ns, pytz.FixedOffset(-300)]', name='Datetime', length=313, freq=None)

In [123]:
macd, macdsignal, macdhist = MACD(df.Close, fastperiod=12, slowperiod=26, signalperiod=9)